# IDyOM

## Introduction and Recap

By now, you are likely familiar with what IDyOM is. As a recap:
- IDyOM is a statistical learning model that estimates the probability of each musical event
- It learns patterns from the current sequence to predict what comes next

The output from IDyOM is a measure known as Information Content. This is calculated as the negative log probability (base 2), which gives us a value measured in bits — the fundamental unit of information from the field of information theory (Shannon, 1948).

Information Content = -log₂(probability)

This measure is useful because it quantifies how surprising or unexpected an event is. One bit of information content means the event had a 50% probability (since -log₂(0.5) = 1). More generally, each additional bit doubles the number of equally likely alternatives that would need to be ruled out. For example, if we had 100 equally likely options, 1 bit would correspond to eliminating 50 of them, leaving 50 remaining possibilities.

As such, **higher information content** means the event was **less probable and more surprising**, while **lower information content** means the event was **more probable and less surprising**. An event with 0 bits of information content would have been completely certain (probability = 1.0), while an event with 3 bits would have had only a 12.5% chance of occurring (probability = 0.125).

It is clear then, that IDyOM is very relevant to understanding human perception of musical events. Information content allows us to simulate the amount of surprise we might feel at every note in a melody, or to summarise this surprise across a whole melody.

## Using IDyOM

We will first need to run a few commands to get IDyOM ready.

This first command simply brings IDyOM into our workspace so we can use it.

In [9]:
from melody_features import idyom_interface as idyom
print(f"IDyOM installed and ready to use: {idyom.is_idyom_installed()}")

IDyOM installed and ready to use: True


Now we need to tell IDyOM what melody we want to get the information content for. We supply this using the *path* to a MIDI file. This is just the filename, surrounded in quotation marks like `"davidwhyatt/Downloads/idyom_test.midi"` 

I have included a test file in this directory, its path is `/workspaces/Part-IB-Expectation-IDyOM/idyom_test.mid`


In [14]:
TEST_MIDI = "/workspaces/Part-IB-Expectation-IDyOM/idyom_test.mid"

In [ ]:
import music21

# Load the MIDI file
score = music21.converter.parse(TEST_MIDI)

# Display the musical notation
score.show()


SubConverterException: Cannot find a path to the 'mscore' file at /usr/bin/mscore3 -- download MuseScore